<a href="https://colab.research.google.com/github/6MrCrazy6/AI_Crypto/blob/main/ai_crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing

## Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import kagglehub

## Importing the dataset

In [ ]:
dataset_path = kagglehub.dataset_download("thedevastator/crypto-trading-and-technical-indicators")
dataset = f"{dataset_path}/11_04_2022_cryptos.csv"

selected_columns = [
    'Volatility',
    'RSI14',
    'MACD Level',
    'MACD Signal',
    'ATR',
    'ADX',
    'MOM',
    'Stoch %K',
    'Stoch %D',
    'AO',
    'CCI20',
    '1W CHG %',
    '1M CHG %'
]

df = pd.read_csv(dataset, usecols=selected_columns)

for col in ['1W CHG %', '1M CHG %']:
    df[col] = df[col].astype(str).str.replace('−', '-', regex=False)
    df[col] = df[col].str.rstrip('%')
    df[col] = pd.to_numeric(df[col], errors='coerce') / 100

for col in selected_columns:
    df[col] = df[col].astype(str).str.extract(r"([-+]?[0-9]*\.?[0-9]+)").astype(float)

future_window = 6
df_full = pd.read_csv(dataset)
df['future_price'] = df_full['Last_x'].shift(-future_window)
df['price_change_pct'] = (df['future_price'] - df_full['Last_x']) / df_full['Last_x']
df.dropna(inplace=True)

df['target'] = (df['price_change_pct'] > 0).astype(int)

X = df[selected_columns].values
y = df['target'].values

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# AI Create

## Training model

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

## Predict the test Resuly

In [ ]:
train_proba = model.predict_proba(X_train)
test_proba  = model.predict_proba(X_test)

## Visualising the Training set results

In [ ]:
train_df = pd.DataFrame({
    "Chance Up (%)":   (train_proba[:, 1] * 100).round(2),
    "Chance Down (%)": (train_proba[:, 0] * 100).round(2)
})

print("=== Training set probabilities ===")
print(train_df.to_string(index=False))

=== Training set probabilities ===
 Chance Up (%)  Chance Down (%)
         61.34            38.66
         49.25            50.75
         50.03            49.97
         50.10            49.90
          9.64            90.36
         49.42            50.58
         27.79            72.21
         61.09            38.91
         59.05            40.95
          4.01            95.99
         34.06            65.94
         68.66            31.34
         50.36            49.64
         54.71            45.29
         52.92            47.08
         53.45            46.55
         63.22            36.78
         44.57            55.43
         48.24            51.76
         50.78            49.22
         70.19            29.81
         55.45            44.55
         48.42            51.58
         56.09            43.91
         49.29            50.71
         69.02            30.98
         59.86            40.14
         50.43            49.57
         45.82            54.18
     

## Visualising the Test set results

In [ ]:
test_df = pd.DataFrame({
    "Chance Up (%)":   (test_proba[:, 1] * 100).round(2),
    "Chance Down (%)": (test_proba[:, 0] * 100).round(2)
})

print("\n=== Test set probabilities ===")
print(test_df.to_string(index=False))


=== Test set probabilities ===
 Chance Up (%)  Chance Down (%)
         51.51            48.49
         61.60            38.40
         43.49            56.51
         60.64            39.36
         44.95            55.05
         53.97            46.03
         41.84            58.16
         47.56            52.44
         67.61            32.39
         54.74            45.26
         54.87            45.13
          0.00           100.00
         57.09            42.91
         69.18            30.82
         15.80            84.20
         59.23            40.77
         57.99            42.01
         72.58            27.42
         50.92            49.08
         55.14            44.86
         50.03            49.97
         38.14            61.86
          5.37            94.63
         49.41            50.59
         24.75            75.25
         47.96            52.04
          0.38            99.62
         35.81            64.19
         25.20            74.80
        

# Upload real data

## Upload and read JSON Data

In [ ]:
import json
from google.colab import files

uploaded = files.upload()

for filename in uploaded:
    with open(filename, 'r') as f:
        real_json_data = json.load(f)

real_df = pd.DataFrame(real_json_data)

KeyboardInterrupt: 